In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import nlppln
import ochre
import os

with nlppln.WorkflowGenerator(working_dir='/Users/jvdzwaan/cwl-working-dir/') as wf:
    wf.load(steps_dir=ochre.cwl_path())
    wf.load(step_file='https://raw.githubusercontent.com/nlppln/ocrevaluation-docker/master/ocrevaluation.cwl')
    
    #print wf.list_steps()

    gt_file = wf.add_input(gt='File')
    ocr_file = wf.add_input(ocr='File')
    xmx = wf.add_input(xmx='string?')

    out_file = wf.ocrevaluation(gt=gt_file, ocr=ocr_file, xmx=xmx)
    character_data, global_data = wf.ocrevaluation_extract(in_file=out_file)

    wf.add_outputs(character_data=character_data)
    wf.add_outputs(global_data=global_data)

    # can be used as a subworkflow
    wf.save(os.path.join(ochre.cwl_path(), 'ocrevaluation-performance-wf.cwl'), mode='wd')

In [ ]:
import nlppln
import ochre
import os

with nlppln.WorkflowGenerator(working_dir='/Users/jvdzwaan/cwl-working-dir/') as wf:
    wf.load(steps_dir=ochre.cwl_path())
    
    print(wf.list_steps())

    gt_dir = wf.add_input(gt='Directory')
    ocr_dir = wf.add_input(ocr='Directory')
    xmx = wf.add_input(xmx='string?')
    performance_file = wf.add_input(out_name='string?', default='performance.csv')

    ocr_files = wf.ls(in_dir=ocr_dir)
    gt_files = wf.ls(in_dir=gt_dir)
    
    character_data, global_data = wf.ocrevaluation_performance_wf(gt=gt_files, ocr=ocr_files, xmx=xmx,
                                                                  scatter=['gt', 'ocr'], scatter_method='dotproduct')
    
    merged = wf.merge_csv(in_files=global_data, name=performance_file)

    wf.add_outputs(performance=merged)

    wf.save(os.path.join(ochre.cwl_path(), 'ocrevaluation-performance-wf-pack.cwl'), mode='pack')

In [ ]:
import nlppln
import ochre
import os

with nlppln.WorkflowGenerator(working_dir='/Users/jvdzwaan/cwl-working-dir/') as wf:
    wf.load(steps_dir=ochre.cwl_path())
    wf.load(step_file='https://raw.githubusercontent.com/nlppln/ocrevaluation-docker/master/ocrevaluation.cwl')
    print(wf.list_steps())

    ocr_dir = wf.add_input(ocr='Directory')
    gt_dir = wf.add_input(gt='Directory')
    datadiv = wf.add_input(datadivision='File')
    div_name = wf.add_input(div_name='string', default='test')
    gt_dir_name = wf.add_input(gt_dir_name='string', default='gs')
    ocr_dir_name = wf.add_input(ocr_dir_name='string', default='ocr')
    fname = wf.add_input(out_name='string?', default='performance.csv')

    ocr_files = wf.select_test_files(datadivision=datadiv, in_dir=ocr_dir, name=div_name)
    gt_files = wf.select_test_files(datadivision=datadiv, in_dir=gt_dir, name=div_name)

    character_data, global_data = wf.ocrevaluation_performance_wf(gt=gt_files, ocr=ocr_files, 
                                                                  scatter=['gt', 'ocr'], 
                                                                  scatter_method='dotproduct')
    
    merged = wf.merge_csv(in_files=global_data, name=performance_file)

    wf.add_outputs(performance=merged)

    wf.save(os.path.join(ochre.cwl_path(), 'ocrevaluation-performance-test-files-wf-pack.cwl'), mode='pack')